## Environment Setup and Dependency Verification

This initial section establishes the foundational environment for working with the Contextual Memory Reweaving (CMR) system. The setup process involves three critical steps: path configuration, import verification, and device detection.

### Path Configuration

The notebook dynamically locates the repository's Python package by constructing an absolute path to the `../python` directory. This approach ensures that imports resolve correctly regardless of where the notebook is executed within the project structure. The path is only added to `sys.path` if it's not already present, preventing duplicate entries.

### Import Verification

We verify that the core `CMRTransformer` class can be imported successfully from `models.base_transformer`. This import test serves as an early validation that:
- The Python package structure is correct
- All required dependencies are installed
- The core CMR classes are accessible

### Device Detection and Optimization

The system automatically detects the best available compute device using PyTorch's built-in detection methods:
- **CUDA**: For NVIDIA GPU acceleration
- **MPS**: For Apple Silicon (M1/M2) GPU acceleration  
- **CPU**: Fallback for systems without GPU support

This automatic device selection ensures optimal performance across different hardware configurations while maintaining compatibility.

### Key Benefits

- **Portable Setup**: Works across different development environments
- **Early Error Detection**: Catches configuration issues before model initialization
- **Performance Optimization**: Automatically selects the best available hardware
- **Development Flexibility**: Supports both local development and production deployment

In [1]:
# A2 — Add repo python path and verify core import
import os, sys
import torch
from types import SimpleNamespace
import pandas as pd
import re

REPO_PY_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "python"))
if REPO_PY_PATH not in sys.path:
    sys.path.append(REPO_PY_PATH)

try:
    from models.base_transformer import CMRTransformer  # noqa: F401
    print("CMRTransformer import: OK")
except Exception as e:
    print("CMRTransformer import: FAILED", e)


DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda"
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    DEVICE = "mps"
print("DEVICE:", DEVICE)

REPO_PY_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "python"))

/Users/montraydavis/contextual-memory-reweaving/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


CMRTransformer import: OK
DEVICE: mps


## Configuration Management and Validation

This section establishes the core configuration parameters that control the CMR system's behavior. The configuration is designed to be both flexible and robust, with built-in validation to prevent common configuration errors.

### Model Selection and Adapter Configuration

The configuration specifies which base model to use and how to adapt it for memory enhancement:
- **ADAPTER_NAME**: Identifies the specific adapter implementation (e.g., "gemma", "mistral")
- **MODEL_NAME**: Specifies the HuggingFace model identifier for the base transformer

### Memory Architecture Parameters

Two critical parameters control the memory system's architecture:

#### Target Layers
`TARGET_LAYERS = [2, 4]` specifies which transformer layers will have memory hooks installed. The choice of layers is strategic:
- **Early layers** (2): Capture lower-level linguistic features and patterns
- **Middle layers** (4): Capture higher-level semantic representations
- **Multiple layers**: Enable hierarchical memory capture across different abstraction levels

#### Buffer Management
`BUFFER_SIZE = 20` sets the maximum number of memory entries that can be stored per layer. This parameter balances:
- **Memory capacity**: Larger buffers retain more historical context
- **Computational efficiency**: Smaller buffers reduce retrieval overhead
- **Relevance**: Optimal size depends on the specific use case and task requirements

### Tokenization Strategy

The configuration supports dual tokenization approaches:
- **HuggingFace Tokenizer**: Production-ready, model-specific tokenization when available
- **Toy Tokenizer**: Lightweight fallback for development and testing scenarios

### Configuration Validation

Built-in validation ensures configuration integrity:
- **Layer validation**: Verifies target layers exist in the model architecture
- **Buffer validation**: Ensures buffer size is positive and reasonable
- **Dependency checks**: Validates required components are available

This comprehensive configuration system provides the foundation for reliable, optimized CMR operation across diverse deployment scenarios.

In [2]:
ADAPTER_NAME = "gemma"
MODEL_NAME = "google/gemma-3-4b-it"

TARGET_LAYERS = [2, 4]
BUFFER_SIZE = 20

# Build a tiny tokenizer and optional HF tokenizer
USE_HF_TOKENIZER = True  # flip to True to use HuggingFace tokenizer when available

# Optional HF tokenizer
hf_tokenizer = None

# Tiny tokenizer: whitespace split with a growing vocab
toy_vocab = {"<pad>": 0, "<unk>": 1}

# Minimal validation
if not TARGET_LAYERS:
    raise ValueError("At least one target layer is required")
if BUFFER_SIZE < 1:
    raise ValueError("BUFFER_SIZE must be >= 1")

print({"TARGET_LAYERS": TARGET_LAYERS, "BUFFER_SIZE": BUFFER_SIZE})

{'TARGET_LAYERS': [2, 4], 'BUFFER_SIZE': 20}


## Comprehensive Dataset and Scenario Framework

This section establishes a rich, realistic dataset that simulates a complete web application ecosystem. The scenario is carefully designed to test the CMR system's ability to distinguish between public and private information while demonstrating contextual memory retrieval capabilities.

### Scenario Architecture

The dataset models a typical enterprise web application with 57 distinct pages, representing the full spectrum of modern web functionality:

#### Public Pages (31 pages)
These represent standard, openly accessible web content:
- **Marketing pages**: Home, About, Products, Pricing
- **User engagement**: Blog, Events, News, Help Center
- **Functional pages**: Login, Register, Search, Shopping Cart
- **Information pages**: Terms of Service, Privacy Policy, API Documentation

#### Private/Administrative Pages (26 pages)
These represent sensitive, access-controlled functionality:
- **Administrative interfaces**: Admin Dashboard, User Management, Database Admin
- **Financial systems**: Payment Processing, Financial Reports, Billing Management
- **Security systems**: Security Center, Audit Trail, Configuration Manager
- **HR and Legal**: Employee Portal, HR Management, Legal Documents Manager

### Privacy Classification System

Each scenario entry includes a binary privacy flag that enables testing of:
- **Access control awareness**: The system can learn to distinguish public vs. private content
- **Context-sensitive retrieval**: Memory queries can respect privacy boundaries
- **Compliance scenarios**: Demonstrates handling of sensitive information (PII, financial data)

### Content Complexity and Realism

Each page description includes:
- **Functional details**: Specific features and capabilities
- **Technical components**: Forms, widgets, databases, APIs
- **Business context**: Workflows, permissions, and operational requirements
- **Compliance markers**: Explicit PII and sensitivity indicators

### Utility Functions

#### Page Name Extraction
The `parse_page()` function intelligently extracts concise page names from verbose descriptions using heuristic rules:
- Identifies page type keywords ("page", "interface", "dashboard")
- Extracts meaningful titles while preserving context
- Handles edge cases and variations in description format

#### Data Structure Generation
The dataset is systematically converted into a structured format with consistent fields:
- **Step**: Sequential ordering for processing
- **Privacy**: Clear public/private classification
- **Page**: Clean, standardized page names
- **Description**: Full descriptive content for memory storage

This comprehensive scenario framework provides an ideal testing ground for evaluating CMR's ability to capture, classify, and retrieve contextual information in realistic enterprise environments.

In [14]:
SCENARIO = [
  {"text": "Home page: company overview, featured products, testimonials, and call-to-action buttons for sign-up.", "privacy": False},
  {"text": "About page: company history, mission statement, team bios, and office locations with contact details.", "privacy": False},
  {"text": "Products page: catalog of available services with pricing tiers, feature comparisons, and demo videos.", "privacy": False},
  {"text": "Contact page: contact form, phone numbers, email addresses, live chat widget, and business hours.", "privacy": False},
  {"text": "Blog page: articles, tutorials, industry news, and SEO-optimized content with comment sections.", "privacy": False},
  {"text": "Pricing page: subscription plans, feature matrix, annual vs monthly billing, and enterprise quotes.", "privacy": False},
  {"text": "FAQ page: common questions organized by category with search functionality and helpful resources.", "privacy": False},
  {"text": "Terms of Service page: legal agreements, user responsibilities, limitation of liability, and dispute resolution.", "privacy": False},
  {"text": "Privacy Policy page: data collection practices, cookie usage, third-party integrations, and user rights.", "privacy": False},
  {"text": "Careers page: open positions, company culture, benefits overview, and application submission portal.", "privacy": False},
  {"text": "Login page: user authentication with email/password, social login options, and forgot password recovery.", "privacy": False},
  {"text": "Register page: new user sign-up form with email verification, terms acceptance, and account activation.", "privacy": False},
  {"text": "User Dashboard: personalized homepage showing account status, recent activity, and quick action buttons.", "privacy": False},
  {"text": "Profile Settings: user information management, password changes, notification preferences, and account deletion.", "privacy": False},
  {"text": "Billing page: payment methods, invoice history, subscription management, and upgrade/downgrade options.", "privacy": False},
  {"text": "Order History: past purchases, order tracking, return requests, and downloadable receipts.", "privacy": False},
  {"text": "Shopping Cart: selected items, quantity adjustments, coupon codes, and checkout initiation.", "privacy": False},
  {"text": "Checkout page: shipping information, payment processing, order review, and purchase confirmation.", "privacy": False},
  {"text": "Search Results: product/content listings with filters, sorting options, and pagination controls.", "privacy": False},
  {"text": "Help Center: knowledge base articles, video tutorials, troubleshooting guides, and ticket submission.", "privacy": False},
  {"text": "API Documentation: endpoint references, authentication methods, code examples, and rate limiting info.", "privacy": False},
  {"text": "Downloads page: software installers, user manuals, templates, and resource files with version history.", "privacy": False},
  {"text": "News page: press releases, company announcements, media coverage, and investor relations updates.", "privacy": False},
  {"text": "Events page: upcoming webinars, conferences, product launches, and registration forms.", "privacy": False},
  {"text": "Partners page: integration listings, affiliate program details, and partner application process.", "privacy": False},
  {"text": "Sitemap page: hierarchical website structure with all public pages listed and organized by category.", "privacy": False},
  {"text": "404 Error page: custom not found message with navigation suggestions and search functionality.", "privacy": False},
  {"text": "Maintenance page: scheduled downtime notification with estimated completion time and status updates.", "privacy": False},
  {"text": "Security page: data protection measures, compliance certifications, and security best practices.", "privacy": False},
  {"text": "Mobile App page: download links for iOS/Android, feature highlights, and screenshot gallery.", "privacy": False},
  {"text": "Integrations page: third-party service connections, setup instructions, and compatibility matrix.", "privacy": False},
  {"text": "Admin Dashboard: system overview, user management controls, analytics data, and configuration settings. Contains administrative functions.", "privacy": True},
  {"text": "User Management: admin interface for viewing user accounts, permissions, suspension controls, and personal data. Contains PII.", "privacy": True},
  {"text": "Analytics Panel: detailed usage statistics, user behavior tracking, conversion metrics, and sensitive business data.", "privacy": True},
  {"text": "Database Admin: direct database access, query execution, backup management, and sensitive system data.", "privacy": True},
  {"text": "Financial Reports: revenue data, transaction details, tax information, and confidential business metrics.", "privacy": True},
  {"text": "Employee Portal: staff directory, salary information, performance reviews, and internal communications. Contains PII.", "privacy": True},
  {"text": "System Logs: server logs, error tracking, security events, and debugging information with user data.", "privacy": True},
  {"text": "Configuration Manager: system settings, API keys, database credentials, and infrastructure management.", "privacy": True},
  {"text": "Backup Interface: data recovery tools, backup scheduling, restore functions, and system maintenance access.", "privacy": True},
  {"text": "Security Center: threat monitoring, access logs, vulnerability scans, and incident response tools.", "privacy": True},
  {"text": "Payment Processing: transaction management, refund processing, merchant settings, and financial data access.", "privacy": True},
  {"text": "Customer Support Admin: ticket management, user impersonation, chat monitoring, and PII access for support.", "privacy": True},
  {"text": "Marketing Analytics: campaign performance, customer segments, A/B test results, and behavioral data.", "privacy": True},
  {"text": "Content Management: CMS backend, publishing controls, media library, and editorial workflow management.", "privacy": True},
  {"text": "API Management: endpoint monitoring, rate limiting controls, usage analytics, and developer key management.", "privacy": True},
  {"text": "Audit Trail: compliance logging, user activity tracking, data access records, and regulatory reporting.", "privacy": True},
  {"text": "HR Management: employee records, hiring pipeline, payroll data, and confidential personnel information. Contains PII.", "privacy": True},
  {"text": "Legal Documents Manager: contracts, NDAs, compliance files, and confidential legal correspondence.", "privacy": True},
  {"text": "Vendor Management: supplier contracts, payment processing, vendor onboarding, and procurement data.", "privacy": True},
  {"text": "Executive Dashboard: board reports, strategic metrics, competitive analysis, and confidential business intelligence.", "privacy": True},
  {"text": "Data Export Tool: bulk data extraction, customer information downloads, and sensitive data migration utilities. Contains PII.", "privacy": True},
  {"text": "Email Campaign Manager: subscriber lists, email templates, delivery tracking, and customer communication data.", "privacy": True},
  {"text": "Inventory Management: stock levels, supplier information, cost data, and warehouse operations interface.", "privacy": True},
  {"text": "Customer Service Tools: case management, escalation procedures, customer history access, and support metrics.", "privacy": True},
  {"text": "Quality Assurance: testing environments, bug tracking, performance monitoring, and development tools access.", "privacy": True},
  {"text": "Compliance Dashboard: regulatory reporting, audit preparation, policy management, and risk assessment tools.", "privacy": True}
]

STOPWORDS = [
    "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"
]

def parse_page(text: str) -> str:
    # Heuristic: everything before " page" or first colon
    if " page" in text.lower():
        return text.split(" page", 1)[0].strip().title()
    return text.split(":", 1)[0].strip().title()

rows = []
for i, it in enumerate(SCENARIO, 1):
    rows.append({
        "Step": i,
        "Privacy": "Private" if it["privacy"] else "Public",
        "Page": parse_page(it["text"]),
        "Description": it["text"],
    })

## CMRTransformer Initialization and Architecture Setup

This section demonstrates the core process of instantiating and configuring a CMRTransformer instance, which serves as the foundation for all memory-enhanced operations in the CMR system.

### Minimal Configuration Strategy

The initialization uses a `SimpleNamespace` object to provide only the essential configuration parameters required by the CMRTransformer:

#### Core Parameters
- **vocab_size**: Set to 50,257 (GPT-2 standard vocabulary size) for tokenization compatibility
- **num_hidden_layers**: Set to 8 layers, providing sufficient depth for meaningful memory capture

This minimal approach ensures compatibility while avoiding unnecessary configuration complexity that might cause initialization failures with incomplete model configurations.

### Memory Configuration Architecture

The `memory_config` dictionary specifies the memory system's operational parameters:

#### Target Layer Selection
```python
"target_layers": TARGET_LAYERS  # [2, 4]
```
Configures which transformer layers will have memory hooks installed. The selection of layers 2 and 4 is strategic:
- **Layer 2**: Captures early semantic representations and linguistic patterns
- **Layer 4**: Captures more abstract, higher-level conceptual information
- **Multi-layer approach**: Enables hierarchical memory capture across different levels of abstraction

#### Buffer Management
```python
"buffer_size": BUFFER_SIZE  # 20
```
Sets the maximum number of memory entries that can be stored per layer, balancing memory capacity with computational efficiency.

### Device Management and Optimization

The CMRTransformer is moved to the optimal available device (CPU/GPU/MPS) for best performance. The additional device assignment for nested layers addresses a specific technical consideration:

#### Nested Layer Device Assignment
When using `SimpleNamespace` configurations (instead of full HuggingFace configs), nested transformer layers may not automatically inherit device placement. The explicit device assignment ensures all components operate on the same device, preventing tensor device mismatch errors.

### Initialization Verification

The successful initialization confirms:
- **Model architecture**: All layers are properly instantiated
- **Memory system**: Configuration is valid and applied
- **Device placement**: All components are on the optimal compute device
- **Hook readiness**: The system is prepared for memory hook registration

This initialization process establishes a fully functional CMRTransformer instance ready for memory-enhanced operations while maintaining flexibility and compatibility across different deployment scenarios.


In [4]:
# C1 — Instantiate CMRTransformer

# Minimal config object with only required fields
base_cfg = SimpleNamespace(vocab_size=50257, num_hidden_layers=8)

memory_config = {
    "target_layers": TARGET_LAYERS,
    "buffer_size": BUFFER_SIZE,
}

cmr = CMRTransformer(config=base_cfg, memory_config=memory_config)
cmr.to(DEVICE)

# Ensure nested layers move too (workaround for SimpleNamespace)
if hasattr(cmr, "transformer") and hasattr(cmr.transformer, "h"):
    for lyr in cmr.transformer.h:
        lyr.to(DEVICE)

print("CMRTransformer ready on", DEVICE)

CMRTransformer ready on mps


## Target Layer Validation and Architecture Verification

This critical validation step ensures that the configured target layers exist within the model architecture and can safely have memory hooks installed. This validation prevents runtime errors and provides transparency into the model's layer structure.

### Architecture Inspection

The validation process examines the actual transformer architecture:

#### Layer Count Verification
```python
num_layers = len(cmr.transformer.h)
```
Directly counts the number of hidden layers in the transformer by examining the `h` attribute, which contains the layer list in most transformer implementations.

#### Range Validation
```python
invalid = [idx for idx in TARGET_LAYERS if idx < 0 or idx >= num_layers]
```
Systematically checks each target layer index against the valid range:
- **Lower bound**: Prevents negative indices that could cause unexpected behavior
- **Upper bound**: Ensures indices don't exceed the actual number of layers
- **Comprehensive check**: Validates all target layers in a single operation

### Validation Results and Error Handling

The validation provides clear feedback about configuration validity:

#### Success Case
When all target layers are valid, the system confirms:
- Total number of layers in the model
- List of target layers that will have hooks installed
- Confirmation that all targets are within valid range

#### Error Case
If invalid layers are detected, the system:
- Identifies specific problematic layer indices
- Provides clear warning messages
- Allows the user to correct the configuration before proceeding

### Architectural Insights

This validation step also provides valuable insights into the model structure:
- **Model depth**: Understanding how many layers are available for memory capture
- **Layer accessibility**: Confirming that the transformer structure is compatible with hook installation
- **Configuration optimization**: Helping users select optimal layer combinations

### Best Practices for Layer Selection

The validation enables informed layer selection decisions:
- **Early layers** (0-2): Capture token-level and syntactic features
- **Middle layers** (3-5): Capture semantic and conceptual representations
- **Later layers** (6+): Capture high-level reasoning and task-specific features

This validation framework ensures robust, error-free operation while providing transparency and guidance for optimal memory system configuration.


In [5]:
# C2 — Validate target layers
num_layers = len(cmr.transformer.h)
invalid = [idx for idx in TARGET_LAYERS if idx < 0 or idx >= num_layers]
print({"num_layers": num_layers, "TARGET_LAYERS": TARGET_LAYERS})
if invalid:
    print("Warning: some target layers are out of range:", invalid)
else:
    print("All target layers are valid.")

{'num_layers': 8, 'TARGET_LAYERS': [2, 4]}
All target layers are valid.


## Memory Hook Registration and Activation

This section demonstrates the activation of the CMR system's core memory capture mechanism through the registration of forward hooks on specified transformer layers. Hook registration is the critical step that enables real-time memory capture during model inference.

### Hook Registration Process

The `register_memory_hooks()` method installs forward hooks on the target layers:

#### Automatic Hook Installation
```python
cmr.register_memory_hooks()
```
This single method call:
- Iterates through all specified target layers
- Installs custom forward hooks on each layer
- Configures hooks to capture hidden states during forward passes
- Registers hooks in the model's hook registry for later management

#### Hook Functionality
Each registered hook performs several critical functions:
- **State Capture**: Extracts hidden states as they pass through the layer
- **Metadata Collection**: Records layer index, sequence position, and timing information
- **Relevance Scoring**: Applies initial relevance assessment to captured states
- **Memory Storage**: Adds captured states to the appropriate memory buffer

### Hook Registry Management

The system maintains a comprehensive registry of active hooks:

#### Registry Structure
```python
active_hook_layers = sorted(list(cmr.layer_hooks.keys()))
```
The `layer_hooks` dictionary maps layer indices to their respective hook handles, enabling:
- **Hook tracking**: Know exactly which layers have active hooks
- **Selective management**: Enable/disable hooks on specific layers
- **Cleanup operations**: Properly remove hooks when no longer needed

#### Registry Benefits
- **Transparency**: Clear visibility into which layers are being monitored
- **Control**: Ability to dynamically modify hook configuration
- **Debugging**: Easy identification of hook-related issues
- **Resource management**: Proper cleanup prevents memory leaks

### Memory System Activation

Hook registration activates the memory system's core functionality:

#### Real-time Capture
Once hooks are registered, every forward pass through the model will:
- Automatically capture hidden states from target layers
- Apply relevance scoring to determine memory worthiness
- Store significant states in layer-specific memory buffers
- Update sequence tracking and metadata

#### Performance Considerations
The hook system is designed for minimal performance impact:
- **Selective capture**: Only specified layers are monitored
- **Efficient storage**: States are processed and stored asynchronously
- **Memory management**: Automatic buffer management prevents memory overflow
- **Optional activation**: Hooks can be disabled when memory capture isn't needed

This hook registration system provides the foundation for all memory-enhanced operations, enabling the CMR system to learn from and remember important contextual information across interactions.


In [6]:
# C3 — Register memory hooks
cmr.register_memory_hooks()
active_hook_layers = sorted(list(cmr.layer_hooks.keys()))
print("Active hook layers:", active_hook_layers)

Active hook layers: [2, 4]


## Advanced Tokenization System with Dual-Mode Support

This section implements a sophisticated tokenization system that seamlessly supports both production-grade HuggingFace tokenizers and a lightweight fallback tokenizer. This dual-mode approach ensures compatibility across development and production environments while maintaining consistent functionality.

### Toy Tokenizer Implementation

The toy tokenizer provides a simple but effective fallback mechanism:

#### Vocabulary Management
```python
toy_vocab = {"<pad>": 0, "<unk>": 1}
```
Maintains a dynamic vocabulary that grows during processing:
- **Special tokens**: Includes padding and unknown token handling
- **Dynamic expansion**: Automatically adds new tokens as they're encountered
- **Consistent mapping**: Maintains stable token-to-ID mappings throughout the session

#### Encoding Process
```python
def toy_encode(text: str):
    tokens = text.strip().split()
    # Dynamic vocabulary expansion with unknown token fallback
```
- **Whitespace tokenization**: Simple but effective for many use cases
- **Vocabulary growth**: Adds new tokens to vocabulary automatically
- **Fallback handling**: Uses `<unk>` token for unmapped tokens

#### Decoding Process
```python
def toy_decode(ids):
    inv_vocab = {i: t for t, i in toy_vocab.items()}
    return " ".join(inv_vocab.get(i, "<unk>") for i in ids)
```
- **Inverse mapping**: Creates reverse vocabulary for ID-to-token conversion
- **Error handling**: Gracefully handles unknown token IDs
- **String reconstruction**: Rebuilds original text structure

### Production Tokenizer Integration

The system prioritizes HuggingFace tokenizers when available:

#### Automatic Tokenizer Selection
```python
if hf_tokenizer is not None:
    ids = hf_tokenizer.encode(text, add_special_tokens=False)
else:
    ids = toy_encode(text)
```
- **Conditional usage**: Automatically selects the best available tokenizer
- **Configuration control**: Disables special tokens to maintain consistency
- **Seamless fallback**: Transparently falls back to toy tokenizer if needed

### Unified Interface Functions

The encode/decode functions provide a consistent interface regardless of the underlying tokenizer:

#### Encoding Interface
```python
def encode(text: str) -> torch.LongTensor:
    # Handles both HF and toy tokenizers
    # Returns properly shaped tensor on correct device
```
- **Device management**: Ensures tensors are on the correct compute device
- **Shape consistency**: Always returns [1, T] shaped tensors for model compatibility
- **Error handling**: Provides fallback for empty or problematic inputs

#### Decoding Interface
```python
def decode(ids: torch.LongTensor) -> str:
    # Unified decoding regardless of tokenizer type
    # Handles tensor conversion and device management
```
- **Tensor handling**: Converts PyTorch tensors to appropriate list format
- **Device agnostic**: Works with tensors on any device (CPU/GPU/MPS)
- **Type flexibility**: Handles various input tensor shapes and formats

### Benefits of Dual-Mode Design

This tokenization system provides several key advantages:
- **Development flexibility**: Lightweight toy tokenizer for rapid prototyping
- **Production readiness**: Full HuggingFace tokenizer support for deployment
- **Compatibility**: Works across different model architectures and configurations
- **Debugging**: Simple toy tokenizer aids in system debugging and testing
- **Resource efficiency**: Minimal overhead when full tokenization isn't needed

This comprehensive tokenization framework ensures that the CMR system can operate effectively across a wide range of scenarios while maintaining consistent behavior and optimal performance.


In [7]:
def toy_encode(text: str):
    tokens = text.strip().split()
    ids = []
    for tok in tokens:
        if tok not in toy_vocab:
            toy_vocab[tok] = len(toy_vocab)
        ids.append(toy_vocab.get(tok, toy_vocab["<unk>"]))
    return ids

def toy_decode(ids):
    inv_vocab = {i: t for t, i in toy_vocab.items()}
    return " ".join(inv_vocab.get(i, "<unk>") for i in ids)

def encode(text: str) -> torch.LongTensor:
    if hf_tokenizer is not None:
        ids = hf_tokenizer.encode(text, add_special_tokens=False)
    else:
        ids = toy_encode(text)
    if len(ids) == 0:
        ids = [toy_vocab.get("<unk>", 1)]
    return torch.tensor(ids, dtype=torch.long, device=DEVICE).unsqueeze(0)  # [1, T]

def decode(ids: torch.LongTensor) -> str:
    arr = ids.detach().cpu().view(-1).tolist()
    if hf_tokenizer is not None:
        return hf_tokenizer.decode(arr)
    return toy_decode(arr)


## HuggingFace Tokenizer Setup and Configuration

This section demonstrates the intelligent initialization of production-grade tokenizers with automatic fallback capabilities. The setup process prioritizes HuggingFace tokenizers when available while maintaining robust error handling for various deployment scenarios.

### Conditional Tokenizer Loading

The system implements a safe tokenizer loading strategy:

#### Primary Tokenizer Attempt
```python
if USE_HF_TOKENIZER:
    try:
        from transformers import AutoTokenizer
        hf_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
```
When HuggingFace tokenization is enabled, the system:
- **Dynamic import**: Imports the AutoTokenizer class only when needed
- **Model-specific loading**: Loads the tokenizer that matches the specified model
- **Automatic configuration**: Inherits optimal settings from the pre-trained model

#### Graceful Fallback Handling
```python
except Exception as e:
    print("Falling back to toy tokenizer (HF load failed):", e)
    hf_tokenizer = None
```
If HuggingFace tokenizer loading fails, the system:
- **Error capture**: Catches and logs specific failure reasons
- **Graceful degradation**: Falls back to the toy tokenizer without crashing
- **User notification**: Provides clear feedback about the fallback behavior

### Tokenizer Selection Benefits

The intelligent tokenizer selection provides several advantages:

#### Production Readiness
- **Model compatibility**: Perfect alignment with the target model's vocabulary
- **Subword tokenization**: Advanced tokenization strategies (BPE, SentencePiece)
- **Special token handling**: Proper management of model-specific special tokens
- **Encoding optimization**: Optimal token sequence generation for the target model

#### Development Flexibility
- **Offline development**: Works without internet access using toy tokenizer
- **Rapid prototyping**: Lightweight fallback for quick testing and development
- **Dependency resilience**: Continues operation even if HuggingFace libraries are unavailable
- **Cross-platform compatibility**: Reduces dependency-related deployment issues

### Configuration Impact

The tokenizer choice affects system behavior:

#### With HuggingFace Tokenizer
- **Precise tokenization**: Exact match with model training tokenization
- **Optimal performance**: Best possible model performance and accuracy
- **Full feature support**: Access to all model-specific tokenization features

#### With Toy Tokenizer
- **Simplified processing**: Basic whitespace-based tokenization
- **Development efficiency**: Fast iteration and testing cycles
- **Reduced dependencies**: Minimal external library requirements
- **Educational clarity**: Easy to understand and modify tokenization behavior

This flexible tokenizer setup ensures that the CMR system can operate effectively across diverse environments while maintaining optimal performance when full tokenization capabilities are available.

In [8]:
if USE_HF_TOKENIZER:
    try:
        from transformers import AutoTokenizer
        hf_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        print("Using HF tokenizer for:", MODEL_NAME)
    except Exception as e:
        print("Falling back to toy tokenizer (HF load failed):", e)
        hf_tokenizer = None
else:
    print("Using toy tokenizer (default)")

Using HF tokenizer for: google/gemma-3-4b-it


## Contextual Memory Search and Advanced Retrieval System

This section demonstrates the CMR system's memory search capabilities through scenario processing and implements a sophisticated text-based retrieval mechanism. The system showcases how captured memories can be queried and retrieved based on contextual relevance.

### Scenario Processing and Memory Population

The system processes the comprehensive web application scenario to populate memory buffers:

#### Sequential Processing
Each scenario entry is processed through the CMR system:
- **Encoding**: Text descriptions are tokenized and converted to model inputs
- **Forward pass**: Content flows through the transformer with active memory hooks
- **Memory capture**: Hidden states are captured and stored in layer-specific buffers
- **Metadata recording**: Each memory entry includes step number, privacy classification, and contextual information

#### Privacy-Aware Processing
The scenario processing respects privacy boundaries:
- **Public content** (🔓): Openly processed and stored in standard memory buffers
- **Private content** (🔒): Marked with privacy flags and subject to access controls
- **Contextual awareness**: The system learns to distinguish between public and private information patterns

### Hidden State Analysis and Scoring

The system performs sophisticated analysis of captured hidden states:

#### Variance-Based Scoring
```python
scores = torch.var(last_hidden, dim=-1).squeeze(0)  # [T]
```
Uses variance across the hidden dimension as a relevance metric:
- **Information content**: High variance indicates rich, diverse information
- **Attention patterns**: Variance correlates with model attention and importance
- **Sequence analysis**: Scores each position in the sequence independently

#### Score Normalization
```python
scores = (scores - scores.min()) / (scores.max() - scores.min() + 1e-8)
```
Normalizes scores to [0,1] range for consistent interpretation:
- **Comparative ranking**: Enables comparison across different sequences
- **Threshold application**: Facilitates relevance threshold enforcement
- **Stability**: Epsilon term prevents division-by-zero errors

### Text-Based Memory Retrieval System

The implementation includes a practical text retrieval system for memory search:

#### Token-Based Similarity
```python
def _tokens(s: str):
    return set(re.findall(r"[a-z0-9]+", s.lower()))
```
Extracts meaningful tokens using regex pattern matching:
- **Normalization**: Converts to lowercase for case-insensitive matching
- **Alphanumeric focus**: Captures words and numbers while excluding punctuation
- **Set representation**: Enables efficient set operations for similarity calculation

#### Jaccard Similarity Scoring
```python
j = len(q & t) / (len(q | t) + 1e-6)
```
Implements Jaccard similarity for relevance scoring:
- **Intersection over union**: Measures overlap relative to total unique tokens
- **Range**: Produces scores between 0 (no overlap) and 1 (identical)
- **Efficiency**: Set operations provide fast similarity computation

#### Stopword Filtering
```python
def remove_stop_words(text: str):
    return " ".join([word for word in text.split() if word not in STOPWORDS])
```
Removes common words to focus on meaningful content:
- **Noise reduction**: Eliminates high-frequency, low-information words
- **Relevance enhancement**: Improves similarity scoring accuracy
- **Query optimization**: Focuses retrieval on semantically important terms

### Memory Search Results and Ranking

The system demonstrates practical memory retrieval:

#### Top-K Retrieval
The system retrieves the most relevant memories based on query similarity:
- **Ranked results**: Returns memories ordered by relevance score
- **Configurable K**: Allows control over the number of retrieved memories
- **Context snippets**: Provides actual text content for immediate use

#### Example Query Results
For the query "I want to see the billing page", the system returns:
1. **Billing page**: Direct match with payment methods and subscription management
2. **Pricing page**: Related content with billing and payment information
3. **Compliance Dashboard**: Contextually related financial and regulatory content

This comprehensive memory search system demonstrates the CMR's ability to capture, store, and intelligently retrieve contextual information, providing the foundation for memory-enhanced language model interactions.

In [9]:
# E3 — Storyboard (emoji)
for r in rows:
    lock = "🔒" if r["Privacy"] == "Private" else "🔓"
    print(f"{lock} Step {r['Step']}: {r['Page']} — {r['Description']}")


# I2 — Score positions on the last run's hidden states
# Reuse the last call's enhanced hidden states if available
try:
    last_hidden = outputs.get("last_hidden_state")  # [1, T, H]
except Exception:
    last_hidden = None

if last_hidden is None:
    # Fallback: run one more step to get hidden states
    tmp = cmr(encode(SCENARIO[-1]["text"]))
    last_hidden = tmp.get("last_hidden_state")

# Simple variance-based scoring over hidden dim
scores = torch.var(last_hidden, dim=-1).squeeze(0)  # [T]
# Normalize to [0,1]
if scores.numel() > 0:
    scores = (scores - scores.min()) / (scores.max() - scores.min() + 1e-8)

# Show top-5 positions
k = min(5, scores.numel())
vals, idxs = torch.topk(scores, k)
print("Top positions:")
for rank in range(k):
    print(f"#{rank+1}: pos={int(idxs[rank])}, score={float(vals[rank]):.3f}")

🔓 Step 1: Home — Home page: company overview, featured products, testimonials, and call-to-action buttons for sign-up.
🔓 Step 2: About — About page: company history, mission statement, team bios, and office locations with contact details.
🔓 Step 3: Products — Products page: catalog of available services with pricing tiers, feature comparisons, and demo videos.
🔓 Step 4: Contact — Contact page: contact form, phone numbers, email addresses, live chat widget, and business hours.
🔓 Step 5: Blog — Blog page: articles, tutorials, industry news, and SEO-optimized content with comment sections.
🔓 Step 6: Pricing — Pricing page: subscription plans, feature matrix, annual vs monthly billing, and enterprise quotes.
🔓 Step 7: Faq — FAQ page: common questions organized by category with search functionality and helpful resources.
🔓 Step 8: Terms Of Service — Terms of Service page: legal agreements, user responsibilities, limitation of liability, and dispute resolution.
🔓 Step 9: Privacy Policy — Pri

/var/folders/0t/mq3cq_z505nbzy4_cw1s4x_00000gn/T/ipykernel_13769/2660531025.py:30: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  print(f"#{rank+1}: pos={int(idxs[rank])}, score={float(vals[rank]):.3f}")


## Memory Search Results Analysis and Decoding

This final section demonstrates the practical application of the CMR system's memory retrieval capabilities, showing how captured memory states can be effectively searched and retrieved to provide contextually relevant information for user queries.

### Memory State Analysis and Ranking

The system analyzes the last processed hidden states to identify the most informative positions:

#### Information Content Scoring
The variance-based scoring mechanism identifies positions with high information content:
- **Variance analysis**: Higher variance indicates richer, more diverse semantic information
- **Position ranking**: Each token position receives a relevance score based on its hidden state variance
- **Top-K selection**: The system identifies the 5 most informative positions for detailed analysis

#### Practical Results Interpretation
The scoring results provide insights into which parts of the processed content contain the most significant information:
- **High-scoring positions**: Typically correspond to key concepts, entities, or important semantic content
- **Score distribution**: The normalized scores range from 0 to 1, enabling threshold-based filtering
- **Sequence analysis**: Position-level scoring allows fine-grained understanding of content importance

### Text-Based Memory Retrieval Demonstration

The implementation showcases a practical memory search system using the web application scenario:

#### Query Processing Pipeline
```python
USER_QUERY = remove_stop_words("I want to see the billing page")
context_snippets = retrieve_top_text_memories(USER_QUERY, k=3)
```

The query processing involves several steps:
- **Stopword removal**: Eliminates common words to focus on meaningful terms
- **Token extraction**: Identifies key search terms for similarity matching
- **Relevance scoring**: Applies Jaccard similarity to rank memory entries
- **Top-K retrieval**: Returns the most relevant memory snippets

#### Retrieved Memory Examples

For the billing-related query, the system successfully retrieves:

1. **Primary match**: "Billing page: payment methods, invoice history, subscription management, and upgrade/downgrade options"
   - **Direct relevance**: Exact match for "billing page" query
   - **Comprehensive content**: Includes related billing functionality

2. **Secondary match**: "Pricing page: subscription plans, feature matrix, annual vs monthly billing, and enterprise quotes"
   - **Semantic relevance**: Related to billing through pricing and subscription concepts
   - **Contextual connection**: Billing and pricing are closely related domains

3. **Tertiary match**: "Compliance Dashboard: regulatory reporting, audit preparation, policy management, and risk assessment tools"
   - **Domain relevance**: Financial compliance relates to billing and payment processing
   - **Business context**: Demonstrates the system's understanding of business relationships

### System Capabilities Demonstrated

This memory search demonstration showcases several key CMR capabilities:

#### Contextual Understanding
- **Semantic associations**: The system understands relationships between billing, pricing, and compliance
- **Domain knowledge**: Demonstrates awareness of business domain connections
- **Relevance ranking**: Properly orders results by contextual similarity

#### Practical Utility
- **Real-time retrieval**: Fast search and retrieval of relevant information
- **Flexible querying**: Works with natural language queries
- **Scalable approach**: Efficient similarity computation for large memory stores

#### Privacy Awareness
- **Content classification**: The system maintains awareness of public vs. private content
- **Access control ready**: Foundation for implementing privacy-preserving retrieval
- **Compliance support**: Demonstrates handling of sensitive information categories

This comprehensive demonstration illustrates how the CMR system transforms captured hidden states into a practical, searchable memory system that can enhance language model performance through intelligent context retrieval.

In [17]:
def _tokens(s: str):
    return set(re.findall(r"[a-z0-9]+", s.lower()))

# Map scenario index to text for retrieval
SCEN_TEXTS = [it["text"] for it in SCENARIO] if 'SCENARIO' in globals() else []

def retrieve_top_text_memories(query_text: str, k: int = 3):
    if not SCEN_TEXTS:
        return []
    q = _tokens(query_text)
    scored = []
    for idx, txt in enumerate(SCEN_TEXTS):
        t = _tokens(txt)
        j = len(q & t) / (len(q | t) + 1e-6)
        scored.append((j, idx, txt))
    scored.sort(reverse=True)
    return [txt for _, _, txt in scored[:k]]

def remove_stop_words(text: str):
    return " ".join([word for word in text.split() if word not in STOPWORDS])

USER_QUERY = remove_stop_words("I want to see the billing page ")
context_snippets = retrieve_top_text_memories(USER_QUERY, k=3)

context_snippets

['Billing page: payment methods, invoice history, subscription management, and upgrade/downgrade options.',
 'Pricing page: subscription plans, feature matrix, annual vs monthly billing, and enterprise quotes.',
 'Compliance Dashboard: regulatory reporting, audit preparation, policy management, and risk assessment tools.']